In [165]:
import numpy as np
import random
import csv
from itertools import combinations
from collections import Counter
from sklearn.cluster import KMeans

In [149]:
def initialize(chunk,n_cluster):
    n = len(chunk)

    # RS
    temp = []
    chunk_X = [p['dimensions'] for p in chunk]
    RS_kmeans = KMeans(n_clusters=n_cluster*5, random_state=0).fit(chunk_X)
    RS_clusters = [clst_id for clst_id,count in Counter(RS_kmeans.labels_).items() if count==1]
    for id,point in enumerate(chunk):
        if RS_kmeans.labels_[id] in RS_clusters:
            chunk_X.pop(id)
            temp.append(point)
    
    # DS
    DS = [Cluster() for i in range(n_cluster)]
    DS_kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(chunk_X)
    for id,point in enumerate(chunk_X):
        clst_id = DS_kmeans.labels_[id]
        DS[clst_id].update(point)

    # CS
    CS = []
    RS = []
    if len(temp) >= n_cluster*5:
        temp_X = [p['dimensions'] for p in temp]
        CS_kmeans = KMeans(n_clusters=n_cluster*5, random_state=0).fit(temp_X)
        CS_clusters = [clst_id for clst_id,count in Counter(CS_kmeans.labels_).items() if count!=1]
        CS = [Cluster() for i in CS_clusters]
        for id,point in enumerate(temp):
            p_label = CS_kmeans.labels_[id]
            if p_label in CS_clusters:
                CS[CS_clusters.index(p_label)].update(point['dimensions'])
            else:
                RS.append(point) 
    else:
        RS = temp
    return DS,CS,RS



In [161]:
class Cluster:
    def __init__(self) -> None:
        self.N = 0
        self.SUM = 0
        self.SUMSQ = 0

    def centroid(self):
        return self.SUM/self.N
    
    def std(self):
        return np.sqrt((self.SUMSQ/self.N)-np.square(self.centroid()))

    def update(self,x):
        self.N += 1
        self.SUM += x
        self.SUMSQ += np.square(x)

    def combine(self,cluster):
        self.N += cluster.N
        self.SUM += cluster.SUM
        self.SUMSQ += cluster.SUMSQ

    def mahalanobis_distance(self,x):
        c = self.centroid()
        std = self.std()
        return np.sqrt(sum(np.square((x-c)/std)))

In [157]:
def create_cluster_mapping(chunk,n_cluster):
    X = [p['dimensions'] for p in chunk]
    kmeans = KMeans(n_clusters=n_cluster*5, random_state=0).fit(X)
    result = {i:[] for i in range(n_cluster*5)}
    for clst_id,point in zip(kmeans.labels_,chunk):
        result[clst_id].append(point)
    return result
    

In [197]:
def initialize(chunk,n_cluster):
    n = len(chunk)
    assigned = []
    unassigned = []
    for clst_id,cluster in create_cluster_mapping(chunk,n_cluster).items():
        if len(cluster)==1:
            unassigned += cluster
        else:
            assigned += cluster

    # DS
    DS = [Cluster() for i in range(n_cluster)]
    assigned_X = [p['dimensions'] for p in assigned]
    DS_kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(assigned_X)
    for clst_id,x in zip(DS_kmeans.labels_,assigned_X):
        DS[clst_id].update(x)

    # CS RS
    CS = []
    RS = []
    if len(unassigned) >= n_cluster*5:
        for clst_id,cluster in create_cluster_mapping(unassigned,n_cluster).items():
            if len(cluster)==1:
                RS += cluster
            else:
                new_cluster = Cluster()
                for point in cluster:
                    new_cluster.update(point["dimensions"])
                CS.append(new_cluster)
    else:
        RS += unassigned     
    return DS,CS,RS

In [159]:
def  intermediate_results(DS,CS,RS):
    return (
        sum([c.N for c in DS]),
        len(CS),
        0+sum([c.N for c in CS]),
        len(RS)
    )

In [280]:
def clst_wise_mdist(clst1,clst2):
    c1 = clst1.centroid()
    c2 = clst2.centroid()
    if np.sum(np.square(clst1.std())) < np.sum(np.square(clst2.std())):
        std = clst1.std()
    else:
        std = clst2.std()
    return np.sqrt(sum(np.square((c1-c2)/std)))

In [340]:
def main(input_path, n_cluster, output_path):
    data = []
    with open(input_path) as file:
        reader = csv.reader(file, delimiter=',')
        for line in reader:
            data.append({'id':line[0],'cluster_gt':line[1],'dimensions':np.array([float(i) for i in line[2:]])})
    random.shuffle(data)
    int_result = []
    thres = 2*np.sqrt(len(data[0]['dimensions']))
    data_chunks = np.array_split(data,5)
    DS,CS,RS = initialize(data_chunks[0],n_cluster)
    int_result.append(intermediate_results(DS,CS,RS))
    count = len(data_chunks[0])
    for i in range(1,5):
        temp = []
        chunk = data_chunks[i]
        for point in chunk:
            x = point["dimensions"]
            assigned = False
            for cluster in DS+CS:
                if cluster.mahalanobis_distance(x) < thres:
                    cluster.update(x)
                    assigned = True
                    break
            if not assigned:
                temp.append(point)
        temp += RS

        if len(temp) >= n_cluster*5:
            RS = []
            for clst_id,cluster in create_cluster_mapping(temp,n_cluster).items():
                if len(cluster)==1:
                    RS += cluster
                else:
                    new_cluster = Cluster()
                    if cluster[0] == cluster[1]:
                        pass
                    for point in cluster:
                        new_cluster.update(point["dimensions"])
                    CS.append(new_cluster)
        else:
            RS = temp
        
        merge_complete = False
        while not merge_complete:
            merge_complete = True
            for clst1_id,clst2_id in combinations(range(len(CS)),2):
                if clst_wise_mdist(CS[clst1_id],CS[clst2_id]) < thres:
                    CS[clst1_id] = CS[clst1_id].combine(CS[clst2_id])
                    CS.pop(clst2_id)
                    merge_complete = False
                    break
        count+= len(chunk)

        if i==4:
            while True:  
                try:
                    for cs_id,cs_cluster in enumerate(CS):
                        for ds_id,ds_cluster in enumerate(DS):
                            if clst_wise_mdist(cs_cluster,ds_cluster) < thres:
                                raise StopIteration
                except StopIteration:
                    DS[ds_id].combine(cs_cluster)
                    CS.pop(cs_id)
                    continue
                break
        int_result.append(intermediate_results(DS,CS,RS))

    result = []
    for point in data:
        for clst_id,cluster in enumerate(DS):
            if cluster.mahalanobis_distance(point['dimensions']) < thres:
                    label = clst_id
                    break
        else:
            label = -1
        result.append((int(point['id']),label))

    result.sort(key=lambda x: x[0])
    with open(output_path,'w') as file:
        file.write("The intermediate results:\n")
        for i,record in enumerate(int_result):
            file.write(f'Round {i+1}: {record[0]},{record[1]},{record[2]},{record[3]}'+'\n')
        file.write('\n')
        file.write('The clustering results:\n')
        for id,label in result:
            file.write(f'{id},{label}\n')
    
    return result
        

In [341]:
input_path, n_cluster, output_path = ('../resource/asnlib/publicdata/hw6_clustering.txt', 9, 'result.txt')
random.seed(0)

In [342]:
result = main(input_path, n_cluster, output_path)

64463


In [328]:
from sklearn.metrics import accuracy_score,completeness_score